# Stochastic Simulation Assignment 3


### **Contributors**  
- **Maarten Stork**  
- **Paul Jungnickel**  
- **Lucas Keijzer**

### **Overview**  
This notebook contains the code and analysis for **Assignment 3 of Stochastic Simulation**. The code follows the order specified in the assignment guidelines and replicates the experiments conducted in the referenced paper. Each section corresponds to (a) key experiment(s).

In [ ]:
from CircleParticleSim import *
import numpy as np
import numpy.random as rand

# 1)

# 2)

# 3)

In [ ]:
rand.seed(42)
run_count = 50
num_particles = 50
num_scales = 20
chain_lens = np.int64(np.round(np.logspace(0,3,num=num_scales, base=10)))
max_cooling_steps = 10000
print(chain_lens)

def run_scales(num_particles):
    arrs = np.zeros([num_scales, run_count]) 

    for i, chain_len in enumerate(chain_lens):
        num_steps = chain_len * max_cooling_steps
        # cooling_steps =  int(num_steps/chain_len)
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=0.2,
                        extra_args = {'markov_chain_len' : chain_len} 
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print(sim.T)

        
    np.save('data/data-3.1-{}-{}.npy'.format(num_particles, num_scales), arrs)

# for num_particles in [16,22,50,1000]:
#     run_scales(num_particles)

arrs = np.load('data/data-3.1-50-20.npy')
arrs = arrs[:]
scales = np.logspace(-2,1,num=num_scales, base=10)
scales1 = 100*scales[:]
print(arrs.shape)
min_energy = np.min(arrs)
arrs -= min_energy
mean = np.mean(arrs, axis=1)
min_energy = 0
plt.plot(scales1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
perc = 5
plt.fill_between(scales1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)
plt.plot(scales1, min_energy*np.ones_like(scales1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('Markov Chain Length')
plt.ylabel('E')
plt.xscale('log')
plt.yscale('log')
# plt.ylim([1,1.001])
plt.legend()

# 4)

In [ ]:
rand.seed(42)
run_count = 100
num_probs = 20
num_particles = 22
probs = np.linspace(0,1,num_probs)

chain_len = 1000
max_cooling_steps = 100
num_steps = chain_len * max_cooling_steps

def run_probs(num_particles):
    arrs = np.zeros([num_probs, run_count]) 

    for i, p in enumerate(probs):
        for j in range(run_count):
            sim  = CircleParticleSim(num_particles, steps=num_steps, seed=rand.randint(0,2**31-1),
                        cooling_schedule = paper_cooling_schedule,
                        step_size_schedule = sqrt_step_size_schedule,
                        random_step_likelihood=p
                        )
            sim.run_simulation(max_cooling_steps, chain_len)
            arrs[i,j] = sim.E

        print('.', end='')

        
    np.save('data/data-4.1-{}-{}.npy'.format(num_particles, num_probs), arrs)


# for num_particles in [16,22,50,1000]:
# for num_particles in [22]:
#     run_probs(num_particles)


arrs = np.load('data/data-4.1-50-20.npy')
arrs = arrs[1:]/2
num_probs = 20
probs = np.linspace(0,1,num_probs)
probs1 = probs[1:]
print(arrs.shape)
mean = np.mean(arrs, axis=1)
min_energy = np.min(arrs)
plt.plot(probs1, mean, label='mean energy + IQR')
# plt.plot(probs, np.percentile(arrs, 5, axis=1))
perc = 25
# plt.fill_between(probs1, np.percentile(arrs, perc, axis=1), np.percentile(arrs, 100-perc, axis=1), alpha=0.3)
merr = 1.96 / np.sqrt(run_count) *np.std(arrs, axis=1)
plt.fill_between(probs1, mean - merr, mean + merr, alpha=0.3)
plt.plot(probs1, min_energy*np.ones_like(probs1), linestyle = ':', color='gray', label='lowest measured energy')
plt.xlabel('P(random step)')
plt.ylabel('E')
plt.legend()
# plt.ylim([2915,2925])
# plt.savefig('plots/4-50nodes.png', dpi=600)
